In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from requests import get
from bs4 import BeautifulSoup
import re

from time import sleep
from random import randint

In [2]:
#Defining columns
Title = []
Version = []
Year = []
Kms = []
Hp = []
Gear_type = []
Fuel_type = []
Fuel_cons = []
Co2 =[]
Prev_owners = []
City = []
Price = []


In [ ]:
url = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
r = requests.get(url)
page = r.content
print(page)

In [ ]:
soup = BeautifulSoup(page,'lxml')
soup.prettify()


In [ ]:
# Selecting the area to obtain data
ads = soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})
print(len(ads))

In [ ]:
# Getting the data and adding it to the df

for element in ads:
    title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
    Title.append(title)
    try:
        version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
    except:
        version = "Na"
    Version.append(version)
    try:
        year = element.find('li', attrs={'data-type':'first-registration'}).text
    except:
        year = "Na"
    Year.append(year)
    try:
        kms = element.find('li', attrs={"data-type" : "mileage"}).text
    except:
        kms = "Na"
    Kms.append(kms)

    try:
        gear_type = element.find('li', attrs={"data-type":"transmission-type"}).text
    except:
        gear_type = "Na"
    Gear_type.append(gear_type)

    try:
        fuel_type = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
    except:
        fuel_type = "Na"
    Fuel_type.append(fuel_type)
    
    try:
        fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
    except:
        fuel_cons = "-"
    Fuel_cons.append(fuel_cons)

    try:
        co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
    except:
        co2 = "Na"
    Co2.append(co2)

    try:
        prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
    except:
        prev_owners = "Na"
    Prev_owners.append(prev_owners)
    try:
        price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
    except:
        price = "Na"
    Price.append(price)

    try:
        hp = element.find('li', text =re.compile(r'CV')).text
    except:
        hp = "Na"
    Hp.append(hp)
    
    try:
        city = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
    except:
        city = "Na"
    City.append(city)
    

    
       

In [ ]:
#Creating the DataFrame
car_ads = pd.DataFrame({"Title":Title,
                        "Version":Version,
                        "Year": Year,
                        "Kms": Kms,
                        "Hp": Hp,
                        "Fuel_type": Fuel_type,
                        "Gear_type": Gear_type,
                        "Fuel_l/100km": Fuel_cons,
                        "CO2_g/Km": Co2,
                        "Previous_owners": Prev_owners,
                        "City_ZIP" : City,
                        "Price_EUR": Price})


In [ ]:
car_ads

In [ ]:
#Data Cleaning

#Year -> Taking only the YYYY
car_ads["Year"] = car_ads["Year"].apply(str).str.strip()
car_ads["Year"] = car_ads["Year"].apply(str).str[3:]

#Kms -> removing \n
car_ads["Kms"] = car_ads["Kms"].apply(str).str.strip()
car_ads["Kms"] = car_ads["Kms"].apply(str).str.strip("km")
car_ads["Kms"] = car_ads["Kms"].apply(str).str.replace(".","")

#Hp
car_ads["Hp"] = car_ads["Hp"].apply(str).str.strip()
car_ads["Hp"] = car_ads["Hp"].apply(str).str.split(pat = "(").str[1]
car_ads["Hp"] = car_ads["Hp"].apply(str).str[:-3]

#Fuel_type
car_ads["Fuel_type"] = car_ads["Fuel_type"].apply(str).str.strip()

#Gear_type
car_ads["Gear_type"] = car_ads["Gear_type"].apply(str).str.strip()

#Fuel_cons
car_ads["Fuel_l/100km"] = car_ads["Fuel_l/100km"].apply(str).str.strip()
car_ads["Fuel_l/100km"] = car_ads["Fuel_l/100km"].apply(str).str.split(pat="/").str[0]
car_ads["Fuel_l/100km"] = car_ads["Fuel_l/100km"].apply(str).str.split().str[0]

#CO2
car_ads["CO2_g/Km"] = car_ads["CO2_g/Km"].apply(str).str.strip()
car_ads["CO2_g/Km"] = car_ads["CO2_g/Km"].apply(str).str.split().str[0]

#Prev_owners
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.strip()
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.split(pat="(").str[1]
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.split(pat=")").str[0]
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.replace("Propietarios","")

#Price
car_ads["Price_EUR"] = car_ads["Price_EUR"].apply(str).str.strip()
car_ads["Price_EUR"] =  car_ads["Price_EUR"].map(lambda x: x.lstrip('€').rstrip(',- \nIVA deducible'))
car_ads["Price_EUR"] =  car_ads["Price_EUR"].apply(str).str.replace(".","").astype(int)

In [ ]:
car_ads

In [ ]:
#Spliting the title into Brand and Model
car_ads[["Brand","Model"]] = car_ads["Title"].str.split(' ', 1, expand = True)

#*****Pending to define long brands and long models******

#Splitting City and ZIP Code

car_ads[["City","ZIP"]] = car_ads["City_ZIP"].str.split(' ',1, expand = True)



In [ ]:
#Analizar url
url_pag1 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
url_pag2 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=2&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
url_pag3 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=3&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"

In [3]:
# Defining pages to start, stop,step No ads
all_pages = np.arange(1,21,1)

In [4]:
# Defining the for Loop for all the pages

for all_page in all_pages:
    
    all_page = requests.get("https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=" + str(all_page) + "&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&")
    all_soup = BeautifulSoup(all_page.text,"lxml")
    all_ads = all_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})
    
    sleep(randint(2,10))
    
    
    for element in all_ads:
        title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
        Title.append(title)
        try:
            version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
        except:
            version = "Na"
        Version.append(version)
        try:
            year = element.find('li', attrs={'data-type':'first-registration'}).text
        except:
            year = "Na"
        Year.append(year)
        try:
            kms = element.find('li', attrs={"data-type" : "mileage"}).text
        except:
            kms = "Na"
        Kms.append(kms)

        try:
            gear_type = element.find('li', attrs={"data-type":"transmission-type"}).text
        except:
            gear_type = "Na"
        Gear_type.append(gear_type)

        try:
            fuel_type = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
        except:
            fuel_type = "Na"
        Fuel_type.append(fuel_type)

        try:
            fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
        except:
            fuel_cons = "-"
        Fuel_cons.append(fuel_cons)

        try:
            co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
        except:
            co2 = "Na"
        Co2.append(co2)

        try:
            prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
        except:
            prev_owners = "Na"
        Prev_owners.append(prev_owners)
        try:
            price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
        except:
            price = "Na"
        Price.append(price)

        try:
            hp = element.find('li', text =re.compile(r'CV')).text
        except:
            hp = "Na"
        Hp.append(hp)
        
        try:
            city = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
        except:
            city = "Na"
        City.append(city)

In [5]:
#Creating the DataFrame
car_ads = pd.DataFrame({"Title":Title,
                        "Version":Version,
                        "Year": Year,
                        "Kms": Kms,
                        "Hp": Hp,
                        "Fuel_type": Fuel_type,
                        "Gear_type": Gear_type,
                        "Fuel_l/100km": Fuel_cons,
                        "CO2_g/Km": Co2,
                        "Previous_owners": Prev_owners,
                        "City_ZIP" : City,
                        "Price_EUR": Price})



In [6]:
car_ads

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_l/100km,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR
0,Citroen Xsara,1.6i 16v Exclusive,\n11/2003\n,\n190.000 km\n,\n80 kW (109 CV)\n,\nGasolina\n,\nManual\n,"\n6,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,18195 CULLAR VEGA,"\n€ 950,-\nIVA deducible\n"
1,SEAT Cordoba,1.9 TDi Sport 90,\n12/2000\n,\n254.000 km\n,\n66 kW (90 CV)\n,\nDiésel\n,\nManual\n,"\n4,9 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,18195 CULLAR VEGA,"\n€ 980,-\nIVA deducible\n"
2,Audi A6,2.3,\n11/1995\n,\n291.000 km\n,\n98 kW (133 CV)\n,\nGasolina\n,\nManual\n,\n0 l/100 km (mixto)\nPuede obtenerse informac...,\n- (g/km)\n,\n- (Propietarios)\n,41003 SEVILLA,"\n€ 1.120,-\n"
3,Peugeot 406,1.9 SRSD,\n06/1998\n,\n150.000 km\n,\n68 kW (92 CV)\n,\nDiésel\n,\nManual\n,"\n6,4 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,28021 MADRID,"\n€ 1.200,-\n"
4,Lancia Dedra,1.8 i.e. LE,\n06/1994\n,\n84.000 km\n,\n78 kW (106 CV)\n,\nGasolina\n,\nManual\n,\n0 l/100 km (mixto)\nPuede obtenerse informac...,\n- (g/km)\n,\n- (Propietarios)\n,26500 CALAHORRA,"\n€ 1.499,-\n"
...,...,...,...,...,...,...,...,...,...,...,...,...
395,Volkswagen Passat CC,1.8 TSI 160cv,\n02/2009\n,\n201.000 km\n,\n118 kW (160 CV)\n,\nGasolina\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,28500 ARGANDA DEL REY,"\n€ 9.990,-\n"
396,Volkswagen Passat CC,1.8 TSI 160cv,\n02/2009\n,\n201.000 km\n,\n118 kW (160 CV)\n,\nGasolina\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,09001 BURGOS,"\n€ 9.990,-\n"
397,Fiat Tipo,1.3 Multijet II Easy,\n02/2019\n,\n30.900 km\n,\n70 kW (95 CV)\n,\nDiésel\n,\nManual\n,"\n4,2 l/100 km (mixto)\nPuede obtenerse inform...",\n- (g/km)\n,\n- (Propietarios)\n,08030 BARCELONA,"\n€ 9.990,-\nIVA deducible\n"
398,Ligier JS 50,Ligier JS 50,\n02/2019\n,\n32.450 km\n,\n6 kW (8 CV)\n,\n- (Combustible)\n,\nAutomático\n,\n- (l/100 km)\n,\n- (g/km)\n,\n- (Propietarios)\n,46910 BENETUSER,"\n€ 9.998,-\nIVA deducible\n"


In [7]:
#Data Cleaning

#Year -> Taking only the YYYY
car_ads["Year"] = car_ads["Year"].apply(str).str.strip()
car_ads["Year"] = car_ads["Year"].apply(str).str[3:]

#Kms -> removing \n
car_ads["Kms"] = car_ads["Kms"].apply(str).str.strip()
car_ads["Kms"] = car_ads["Kms"].apply(str).str.strip("km")
car_ads["Kms"] = car_ads["Kms"].apply(str).str.replace(".","")

#Hp
car_ads["Hp"] = car_ads["Hp"].apply(str).str.strip()
car_ads["Hp"] = car_ads["Hp"].apply(str).str.split(pat = "(").str[1]
car_ads["Hp"] = car_ads["Hp"].apply(str).str[:-3]

#Fuel_type
car_ads["Fuel_type"] = car_ads["Fuel_type"].apply(str).str.strip()

#Gear_type
car_ads["Gear_type"] = car_ads["Gear_type"].apply(str).str.strip()

#Fuel_cons
car_ads["Fuel_l/100km"] = car_ads["Fuel_l/100km"].apply(str).str.strip()
car_ads["Fuel_l/100km"] = car_ads["Fuel_l/100km"].apply(str).str.split(pat="/").str[0]
car_ads["Fuel_l/100km"] = car_ads["Fuel_l/100km"].apply(str).str.split().str[0]

#CO2
car_ads["CO2_g/Km"] = car_ads["CO2_g/Km"].apply(str).str.strip()
car_ads["CO2_g/Km"] = car_ads["CO2_g/Km"].apply(str).str.split().str[0]

#Prev_owners
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.strip()
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.split(pat="(").str[1]
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.split(pat=")").str[0]
car_ads["Previous_owners"] = car_ads["Previous_owners"].apply(str).str.replace("Propietarios","")

#Price
car_ads["Price_EUR"] = car_ads["Price_EUR"].apply(str).str.strip()
car_ads["Price_EUR"] =  car_ads["Price_EUR"].map(lambda x: x.lstrip('€').rstrip(',- \nIVA deducible'))
car_ads["Price_EUR"] =  car_ads["Price_EUR"].apply(str).str.replace(".","").astype(int)

In [8]:
car_ads

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_l/100km,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR
0,Citroen Xsara,1.6i 16v Exclusive,2003,190000,109,Gasolina,Manual,"6,9",-,,18195 CULLAR VEGA,950
1,SEAT Cordoba,1.9 TDi Sport 90,2000,254000,90,Diésel,Manual,"4,9",-,,18195 CULLAR VEGA,980
2,Audi A6,2.3,1995,291000,133,Gasolina,Manual,0,-,,41003 SEVILLA,1120
3,Peugeot 406,1.9 SRSD,1998,150000,92,Diésel,Manual,"6,4",-,,28021 MADRID,1200
4,Lancia Dedra,1.8 i.e. LE,1994,84000,106,Gasolina,Manual,0,-,,26500 CALAHORRA,1499
...,...,...,...,...,...,...,...,...,...,...,...,...
395,Volkswagen Passat CC,1.8 TSI 160cv,2009,201000,160,Gasolina,Automático,-,-,,28500 ARGANDA DEL REY,9990
396,Volkswagen Passat CC,1.8 TSI 160cv,2009,201000,160,Gasolina,Automático,-,-,,09001 BURGOS,9990
397,Fiat Tipo,1.3 Multijet II Easy,2019,30900,95,Diésel,Manual,"4,2",-,,08030 BARCELONA,9990
398,Ligier JS 50,Ligier JS 50,2019,32450,8,- (Combustible),Automático,-,-,,46910 BENETUSER,9998


In [9]:
#Spliting the title into Brand and Model
car_ads[["Brand","Model"]] = car_ads["Title"].str.split(' ', 1, expand = True)

#*****Pending to define long brands and long models******

#Splitting City and ZIP Code

car_ads[["ZIP","City"]] = car_ads["City_ZIP"].str.split(' ',1, expand = True)

In [10]:
car_ads

,Title,Version,Year,Kms,Hp,Fuel_type,Gear_type,Fuel_l/100km,CO2_g/Km,Previous_owners,City_ZIP,Price_EUR,Brand,Model,ZIP,City
0,Citroen Xsara,1.6i 16v Exclusive,2003,190000,109,Gasolina,Manual,"6,9",-,,18195 CULLAR VEGA,950,Citroen,Xsara,18195,CULLAR VEGA
1,SEAT Cordoba,1.9 TDi Sport 90,2000,254000,90,Diésel,Manual,"4,9",-,,18195 CULLAR VEGA,980,SEAT,Cordoba,18195,CULLAR VEGA
2,Audi A6,2.3,1995,291000,133,Gasolina,Manual,0,-,,41003 SEVILLA,1120,Audi,A6,41003,SEVILLA
3,Peugeot 406,1.9 SRSD,1998,150000,92,Diésel,Manual,"6,4",-,,28021 MADRID,1200,Peugeot,406,28021,MADRID
4,Lancia Dedra,1.8 i.e. LE,1994,84000,106,Gasolina,Manual,0,-,,26500 CALAHORRA,1499,Lancia,Dedra,26500,CALAHORRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Volkswagen Passat CC,1.8 TSI 160cv,2009,201000,160,Gasolina,Automático,-,-,,28500 ARGANDA DEL REY,9990,Volkswagen,Passat CC,28500,ARGANDA DEL REY
396,Volkswagen Passat CC,1.8 TSI 160cv,2009,201000,160,Gasolina,Automático,-,-,,09001 BURGOS,9990,Volkswagen,Passat CC,09001,BURGOS
397,Fiat Tipo,1.3 Multijet II Easy,2019,30900,95,Diésel,Manual,"4,2",-,,08030 BARCELONA,9990,Fiat,Tipo,08030,BARCELONA
398,Ligier JS 50,Ligier JS 50,2019,32450,8,- (Combustible),Automático,-,-,,46910 BENETUSER,9998,Ligier,JS 50,46910,BENETUSER
